In [1]:
import pandas as pd
from decision_tree import DT
from sklearn.metrics import confusion_matrix

data = pd.read_csv('./tests_datasets/testing.csv')

data_train = data.sample(frac=0.7, random_state=43)
data_test = data[~data.index.isin(data_train.index)]
# print(data_train.head())
# print(data_train.shape)
# print(data_test.shape)
# print(data.shape)
# data_test = data[data.index.isin(data_train)]
X_train = data_train.drop(columns='Clase').to_numpy()
Y_train = data_train['Clase'].to_numpy()

X_test = data_test.drop(columns='Clase').to_numpy()
Y_test = data_test['Clase'].to_numpy()



In [2]:
dt = DT(X_train,Y_train)


In [3]:
a = confusion_matrix(Y_test,dt.predict(X_test))

total_elems = a[0][0] + a[0][1] + a[1][0] + a[1][1]

print( (a[0][0] + a[1][1]) / total_elems )


0.92
